In [2]:
# Libs Necessárias
import numpy as np
import pandas as pd
import pandas_datareader as web

# Libs para gráficos
import matplotlib.pyplot as plt

# Libs para utilizar o Plotly
import plotly.express as px
import plotly.graph_objects as Dash

!pip install yfinance
import yfinance as yf

# Libs para uso de Machine Learning do Keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Ignorando avisos
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

     |████████████████████████████████| 6.4 MB 8.3 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [8]:
Base_Dados = pd.read_csv('/content/BTC-USD.csv')
Base_Dados

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-10-01,387.427002,411.697998,289.295990,338.321014,338.321014,902994450
1,2014-11-01,338.649994,457.092987,320.626007,378.046997,378.046997,659733360
2,2014-12-01,378.248993,384.037994,304.231995,320.192993,320.192993,553102310
3,2015-01-01,320.434998,320.434998,171.509995,217.464005,217.464005,1098811912
4,2015-02-01,216.867004,265.610992,212.014999,254.263000,254.263000,711518700
...,...,...,...,...,...,...,...
82,2021-08-01,41460.843750,50482.078125,37458.003906,47166.687500,47166.687500,1014674184428
83,2021-09-01,47099.773438,52853.765625,39787.609375,43790.894531,43790.894531,1102139678824
84,2021-10-01,43816.742188,66930.390625,43320.023438,61318.957031,61318.957031,1153077903534
85,2021-11-01,61320.449219,68789.625000,53569.765625,57005.425781,57005.425781,1053270271383


In [9]:
# Plot para verificar preço de fechamento
# Chamando o gráfico
fig = px.line( Base_Dados, y='Close', x= 'Date' )
fig.show()

In [10]:
 # --- Gráficos com as médias
# Gerar a média movel do fechamento das ações
Media_Movel = Base_Dados['Close'].rolling(5).mean()
Media_Movel_Tendencia = Base_Dados['Close'].rolling(30).mean()

# Definindo uma figura
Figura = Dash.Figure()

# Incluindo o Eixo no Gráfico - Abertura
Figura.add_trace(Dash.Scatter(x = Base_Dados.index, y = Base_Dados.Close, 
                    mode='lines',
                    name='Fechamento',
                    marker_color = '#FF7F0E'))

# Incluindo o Eixo no Gráfico - Maior
Figura.add_trace(Dash.Scatter(x = Base_Dados.index, y = Media_Movel,
                    mode='lines',
                    name='Média Móvel', opacity=0.5,
                    marker_color = '#2CA02C',
                    ))

# Incluindo o Eixo no Gráfico - Menor
Figura.add_trace(Dash.Scatter(x = Base_Dados.index, y = Media_Movel_Tendencia,
                    mode='lines',
                    name='Tendência', opacity=0.5,
                    marker_color = '#D62728'))

# Modificando o Layout do Gráfico
Figura.update_layout(
    title='Histórico de Preço', # Titulo
    titlefont_size = 28, # Tamanho da Fonte
    
    # Parametros para mexer no eixo X
    xaxis = dict( 
        title='Período Histórico', # Titulo do Eixo x
        titlefont_size=16, # Tamanho fonte do Titulo 
        tickfont_size=14), # Tamanho da fonte do eixo 

    # Tamanho do Grafico
    height = 500, 
    
    # Parametros para mexer no eixo y
    yaxis=dict(
        title='Preço Bitcoint', # Titulo do Eixo y
        titlefont_size=16, # Tamanho fonte do Titulo 
        tickfont_size=14), # Tamanho da fonte do eixo 
    
    # Parametros para mexer na legenda
    legend=dict(
        y=1, x=1, # Posição da Legenda
        bgcolor='rgba(255, 255, 255, 0)', # Cor de fundo
        bordercolor='rgba(255, 255, 255, 0)')) # Cor da Bornda

# Mostrando o Gráfico
Figura.show()

In [11]:
# Estatisticas
Base_Dados.describe()

,Open,High,Low,Close,Adj Close,Volume
count,87.000000,87.000000,87.000000,87.000000,87.000000,8.700000e+01
mean,10390.723732,12812.228108,8794.545089,10929.657121,10929.657121,4.335727e+11
std,15277.684861,18448.292396,12756.902002,15739.924187,15739.924187,5.590900e+11
min,216.867004,247.804001,171.509995,217.464005,217.464005,5.531023e+08
25%,592.737519,666.893005,551.072479,617.208007,617.208007,2.484313e+09
50%,6318.140137,6965.060059,4164.049805,6404.000000,6404.000000,1.583595e+11
75%,10138.020996,12012.790528,8976.270508,10227.349609,10227.349609,6.926483e+11
max,61320.449219,68789.625000,53569.765625,61318.957031,61318.957031,2.267153e+12


In [12]:
# --- Pre processamento

# Função para escolanemnto
from sklearn.preprocessing import MinMaxScaler

# Chamando a Função
Funcao_MinMAx = MinMaxScaler( feature_range=(0,1) )

# Criando uma copia
Dados_Treino = Base_Dados.filter(['Close'])

# Aplicando a função
Dados_Treino_Escalados = Funcao_MinMAx.fit_transform( Dados_Treino )

# VErificando
Dados_Treino_Escalados[0:5]

array([[0.00197797],
       [0.00262814],
       [0.00168128],
       [0.        ],
       [0.00060226]])

In [13]:
# --- Separação dos dados de treino e teste

# Listas para receber os dados
x_treinamento = []
y_treinamento = []

# Loop para separar os dados de treino e teste
# Nesse Loop vamos seprar os dados em blocos de 60 valores
for Loop in range( 60, len(Dados_Treino_Escalados) ):

    # Separando os dados de treinamento x
    Filtrando_Amostra_Treinamento_x = Dados_Treino_Escalados[ Loop-60 : Loop, 0 ]
    x_treinamento.append( Filtrando_Amostra_Treinamento_x )

    # Separando os dados de treinamento y
    Filtrando_Amostra_Treinamento_y = Dados_Treino_Escalados[Loop, 0] 
    y_treinamento.append( Filtrando_Amostra_Treinamento_y )
    
# Transformando as listas em Array
x_treinamento, y_treinamento = np.array(x_treinamento), np.array(y_treinamento) 

# Convertendo o array para Matriz
x_treinamento = np.reshape(x_treinamento, (x_treinamento.shape[0], x_treinamento.shape[1], 1))

# Verificando a demisão da nossa matriz
x_treinamento.shape

(27, 60, 1)

In [14]:
# --- Treinamento do Modelo

# Definindo a função do Keras
# Essa função é uma pilha linear de camadas do Keras
Modelo = Sequential()

# Adicinando as camadas e parametros para nossa rede neural
# Treinamento da Rede Neural Recorrente

# LSTM - Long Short-Term Memory
Modelo.add(LSTM(50, return_sequences = True, 
                input_shape = (x_treinamento.shape[1], 1)))
Modelo.add(LSTM(50, return_sequences = False))

# Adicionando as camadas na rede neural
Modelo.add(Dense(25))
Modelo.add(Dense(1))
Modelo.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Treinando o modelo
Modelo.fit(x_treinamento, y_treinamento, batch_size = 1, epochs = 10)

Epoch 1/10
27/27 [==============================] - 4s 29ms/step - loss: 0.1166
Epoch 2/10
27/27 [==============================] - 1s 28ms/step - loss: 0.0416
Epoch 3/10
27/27 [==============================] - 1s 29ms/step - loss: 0.0382
Epoch 4/10
27/27 [==============================] - 1s 29ms/step - loss: 0.0380
Epoch 5/10
27/27 [==============================] - 1s 28ms/step - loss: 0.0357
Epoch 6/10
27/27 [==============================] - 1s 29ms/step - loss: 0.0351
Epoch 7/10
27/27 [==============================] - 1s 28ms/step - loss: 0.0368
Epoch 8/10
27/27 [==============================] - 1s 28ms/step - loss: 0.0419
Epoch 9/10
27/27 [==============================] - 1s 30ms/step - loss: 0.0374
Epoch 10/10
27/27 [==============================] - 1s 28ms/step - loss: 0.0429


In [15]:
import math

# Arredondando o numero para cima usando o 'math.ceil'
Dados_Fechamento_Valores_Tamanho = math.ceil( len(Dados_Treino) * .8)

Dados_Fechamento_Valores_Tamanho

70

In [16]:
# Definindo amostra para ser testada
Dados_Teste = Dados_Treino_Escalados[Dados_Fechamento_Valores_Tamanho - 60: , :]

# Lista para recever os dados de teste
x_teste = []

# Lista com os dados de teste
y_Teste = Dados_Treino_Escalados[Dados_Fechamento_Valores_Tamanho:, :]

# Loop para fixar amostra para teste
for Loop in range (60, len(Dados_Teste)):
    x_teste.append(Dados_Teste[Loop - 60:Loop, 0])

# Transformando os dados em um array
x_teste = np.array(x_teste)

# Convertendo o array para Matriz
x_teste = np.reshape(x_teste, (x_teste.shape[0], x_teste.shape[1], 1))

# Aplicando as Previsões
Previsoes = Modelo.predict(x_teste)

# Calculando o erro quadrático médio
rsme = np.sqrt(np.mean(Previsoes - y_Teste) ** 2)
print('Erro Quadrático Médio:', rsme )

# Invertendo para escalas reais 
Previsoes = Funcao_MinMAx.inverse_transform(Previsoes)

Erro Quadrático Médio: 0.022003512307173418


In [17]:
# Criando a base para verificar o real x modelo
Validação = Dados_Treino[Dados_Fechamento_Valores_Tamanho:]

# Atribuindo as previsões no DataSet
Validação['Previsões'] = Previsoes

In [18]:
# Criando um Gráfico Dinâmico
# No gráfico é possivel filtrar pela legenda a informação
# Utlizar zoons

# Definindo uma figura
Figura = Dash.Figure()

# Incluindo o Eixo no Gráfico - Fechamento
Figura.add_trace(Dash.Scatter(x = Validação.index, y = Validação.Close, 
                    mode='lines',
                    name='Fechamento',
                    marker_color = '#FF7F0E',
                    ))

# Incluindo o Eixo no Gráfico - Previsão
Figura.add_trace(Dash.Scatter(x = Validação.index, y = Validação.Previsões,
                    mode='lines',
                    name='Previsão',
                    marker_color = '#2CA02C',
                    ))

# Modificando o Layout do Gráfico
Figura.update_layout(
    title='Realizado vs Modelo', # Titulo
    titlefont_size = 28, # Tamanho da Fonte
    
    # Parametros para mexer no eixo X
    xaxis = dict( 
        title='Período Histórico', # Titulo do Eixo x
        titlefont_size=16, # Tamanho fonte do Titulo 
        tickfont_size=14), # Tamanho da fonte do eixo 

    # Tamanho do Grafico
    height = 500, 
    
    # Parametros para mexer no eixo y
    yaxis=dict(
        title='Preço do Bitcoin', # Titulo do Eixo y
        titlefont_size=16, # Tamanho fonte do Titulo 
        tickfont_size=14), # Tamanho da fonte do eixo 
    
    # Parametros para mexer na legenda
    legend=dict(
        y=1, x=1, # Posição da Legenda
        bgcolor='rgba(255, 255, 255, 0)', # Cor de fundo
        bordercolor='rgba(255, 255, 255, 0)')) # Cor da Bornda

# Mostrando o Gráfico
Figura.show()